In [1]:
import gym
import numpy as np
import random
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
env = gym.make('FrozenLake-v0')
#env = gym.wrappers.Monitor(env, 'tmp/FrozenLake-v0-test-network')

[2017-03-20 02:25:28,718] Making new env: FrozenLake-v0


In [3]:
# set learning parameters
n_action      = env.action_space.n
n_state       = env.observation_space.n

epsilon       = 0.1
learning_rate = 0.1
gamma         = 0.99
n_episode     = 10000

In [4]:
sess_config = tf.ConfigProto()
sess_config.gpu_options.allow_growth = True
sess_config.allow_soft_placement=True 
sess_config.log_device_placement=True

In [5]:
inputs = tf.placeholder(shape=[1,16],dtype=tf.float32)
W = tf.Variable(tf.random_uniform([16,4],0,0.01))
Q = tf.matmul(inputs,W)
a_predict = tf.argmax(Q,1)

Q_next = tf.placeholder(shape=[1,4],dtype=tf.float32)
loss = tf.reduce_sum(tf.square(Q_next - Q))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss)

In [6]:
j_list = []
r_list = []

init_op = tf.global_variables_initializer()
sess = tf.Session(config=sess_config)
sess.run(init_op)

for i in xrange(n_episode):
    s = env.reset()
    r_all = 0
    d = False
    j = 0
    while j < 99:
        j += 1
        a, q_all = sess.run([a_predict, Q], feed_dict={inputs:np.identity(16)[s:s+1]})
        
        if np.random.rand(1) < epsilon:
            a[0] = env.action_space.sample()
        s_next, r, d, _ = env.step(a[0])
        q_next = sess.run(Q, feed_dict={inputs:np.identity(16)[s_next:s_next+1]})
        
        max_q_next = np.max(q_next)
        q_target = q_all
        q_target[0, a[0]] = r + gamma*max_q_next
        
        _ = sess.run(train_op,feed_dict={inputs:np.identity(16)[s:s+1],Q_next:q_target})        
        
        r_all += r
        s = s_next
        if d == True:
            e = 1./((i/50) + 10)
            break
    j_list.append(j)
    r_list.append(r_all)
    if i != 0 and (i+1) % 1000 == 0:
        print "At iteration " + str(i+1) + ": Percent of successful episodes: " + str(sum(r_list)/(i+1)) + "%"

At iteration 1000: Percent of succesful episodes: 0.199199199199%
At iteration 2000: Percent of succesful episodes: 0.294147073537%
At iteration 3000: Percent of succesful episodes: 0.322774258086%
At iteration 4000: Percent of succesful episodes: 0.330582645661%
At iteration 5000: Percent of succesful episodes: 0.336267253451%
At iteration 6000: Percent of succesful episodes: 0.341056842807%
At iteration 7000: Percent of succesful episodes: 0.342906129447%
At iteration 8000: Percent of succesful episodes: 0.346418302288%
At iteration 9000: Percent of succesful episodes: 0.347705300589%
At iteration 10000: Percent of succesful episodes: 0.351535153515%


In [10]:
print "Percent of succesful episodes: " + str(sum(r_list)/n_episode) + "%"

Percent of succesful episodes: 0.228%
